<a href="https://colab.research.google.com/github/MathanRajOlaganathan/Coursera_Capstone/blob/master/Segmenting_and_Clustering_Neighborhoods_in_toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# importing libraries
import pandas as pd
import numpy as np
import re
import json
from pandas.io.json import json_normalize

Read the data from the following wiki page
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [0]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [0]:
table_list = pd.read_html(url)

In [10]:
len(table_list)

3

The wiki page has 3 tables and the first table has list of postal codes in canada.Lets check the first table.

In [14]:
table_list[0][:5]

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


Lets convert the list in to dataframe

In [0]:
df = pd.DataFrame(table_list[0])

In [16]:
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [18]:
df.shape

(180, 3)

The inital dataframe has 180 observations

Lets remove the rows for which the  borough  is Not assigned.

In [0]:
df_rem = df[df['Borough']!='Not assigned'].reset_index(drop=True)

In [36]:
df_rem.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [37]:
df_rem.shape

(103, 3)

After removing the not assigned borough, there are 103 observations

Lets replace / with comma in Neighborhood column

In [0]:
df_rem['Neighborhood'] = df_rem['Neighborhood'].map(lambda x: re.sub(r"\s[/]", ",", x))


In [44]:
df_rem.shape

(103, 3)

Lets rename the Postal code column name

In [0]:
df_rem.rename(columns={'Postal code':'PostalCode'},inplace=True)

In [49]:
df_rem.columns


Index(['PostalCode', 'Borough', 'Neighborhood'], dtype='object')

In [50]:
df_rem.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [54]:
df_rem[df_rem['PostalCode']=='M5G']

,PostalCode,Borough,Neighborhood
24,M5G,Downtown Toronto,Central Bay Street


In [51]:
df_rem['PostalCode'].value_counts()

M1R    1
M4V    1
M7R    1
M4R    1
M4K    1
      ..
M6B    1
M7Y    1
M9P    1
M9N    1
M1K    1
Name: PostalCode, Length: 103, dtype: int64

In [52]:
df_rem.shape

(103, 3)

In [0]:
43.69,-79.311

In [101]:
!pip install pgeocode

**Problem 2 -  get the latitude and the longitude coordinates of each neighborhood**

In [0]:
 import pgeocode

nomi = pgeocode.Nominatim('ca')

In [0]:
Latitude=[]
Longitude=[]
for i in range(df_rem.shape[0]):
  # address=str(df_rem['PostalCode'][i] +" "+ df_rem['Borough'][i] + " "+ 'canada')
  try:
    Latitude.append(nomi.query_postal_code(df_rem['PostalCode'][i]).latitude)
    Longitude.append(nomi.query_postal_code(df_rem['PostalCode'][i]).longitude)
  except (AttributeError,RuntimeError, TypeError, NameError):
    print('error')

In [113]:
len(Latitude),len(Longitude)

(103, 103)

In [0]:
df_rem['Latitude']=Latitude

In [0]:
df_rem['Longitude']=Longitude

In [119]:
df_rem.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889


**Problem3 - Explore and cluster the neighborhoods in Toronto.**

In [0]:
import requests
import folium
from sklearn.cluster import KMeans

Filter the Dataframe to include only the boroughs contain toronto

In [0]:
toronto_df = df_rem[df_rem['Borough'].str.contains('Toronto')].reset_index(drop=True)

In [133]:
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756
4,M4E,East Toronto,The Beaches,43.6784,-79.2941


FourSqaure Credentials

In [120]:
CLIENT_ID = 'FEVOBQA4RKSCY1YIPFWXESFZ530X3TL03CL0QKL4SACUY450' # your Foursquare ID
CLIENT_SECRET = '5FAMDZNCGGBNFPTGDLXMWAX3PE30FZ5GGYIARJHJF153D3AL' # your Foursquare Secret
VERSION = '20200406'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FEVOBQA4RKSCY1YIPFWXESFZ530X3TL03CL0QKL4SACUY450
CLIENT_SECRET:5FAMDZNCGGBNFPTGDLXMWAX3PE30FZ5GGYIARJHJF153D3AL


Function to explore the neighborhood

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [134]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst  Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town

In [135]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.6555,-79.3626,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.6555,-79.3626,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.6555,-79.3626,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.6555,-79.3626,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.6555,-79.3626,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


Let's check how many venues were returned for each neighborhood

In [136]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,30,30,30,30,30,30
"Brockton, Parkdale Village, Exhibition Place",30,30,30,30,30,30
Business reply mail Processing CentrE,15,15,15,15,15,15
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",30,30,30,30,30,30
Central Bay Street,30,30,30,30,30,30
Christie,12,12,12,12,12,12
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",30,30,30,30,30,30
Davisville,23,23,23,23,23,23


In [138]:
print('There are {} unique venue categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 173 unique venue categories.


Lets apply one hot encoding to venu categories

In [171]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
features = list(toronto_onehot.columns)
features.remove('Neighborhood')
fixed_columns = ['Neighborhood'] + features
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,...,Photography Studio,Pizza Place,Playground,Plaza,Poke Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [173]:
toronto_onehot.shape

(785, 173)

let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [174]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,...,Photography Studio,Pizza Place,Playground,Plaza,Poke Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.033333,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.066667,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.0,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.066667,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Business reply mail Processing CentrE,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.133333,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.133333,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.0,0.100000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

In [175]:
toronto_grouped.shape

(39, 173)

Function to sort the venues in descending order.

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

let's create the new dataframe and display the top 10 venues for each neighborhood.

In [177]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Café,Restaurant,Farmers Market,Beer Bar,Gastropub,Basketball Stadium,Museum,Lounge
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Gift Shop,Thrift / Vintage Store,Café,Breakfast Spot,Sandwich Place,Supermarket,Diner,Cocktail Bar,Pet Store
2,Business reply mail Processing CentrE,Coffee Shop,Restaurant,Yoga Studio,Italian Restaurant,Salad Place,Breakfast Spot,Bookstore,Sushi Restaurant,Bank,Japanese Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Restaurant,Coffee Shop,Park,Italian Restaurant,Yoga Studio,Brewery,Sandwich Place,Café,Ramen Restaurant,Caribbean Restaurant
4,Central Bay Street,Coffee Shop,Japanese Restaurant,Tea Room,American Restaurant,Bubble Tea Shop,Restaurant,Italian Restaurant,Café,Poke Place,Chinese Restaurant


Cluster Neighborhoods

In [178]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 0, 2, 2, 2, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [180]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,2,Coffee Shop,Breakfast Spot,Restaurant,Food Truck,Mexican Restaurant,Distribution Center,Pub,Electronics Store,Event Space,Italian Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,2,Gym,Italian Restaurant,Gastropub,Hobby Shop,Bubble Tea Shop,Creperie,Burrito Place,Indian Restaurant,Ramen Restaurant,Distribution Center
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783,2,Coffee Shop,Café,Theater,Clothing Store,Mexican Restaurant,Electronics Store,Sandwich Place,Restaurant,Burger Joint,Burrito Place
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756,2,Gastropub,Italian Restaurant,Japanese Restaurant,Coffee Shop,Café,Restaurant,Beer Bar,Gym,Food Truck,Hostel
4,M4E,East Toronto,The Beaches,43.6784,-79.2941,2,Pub,Gastropub,Trail,Coffee Shop,Bakery,Health Food Store,Yoga Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


Lets create map for the clusters

In [0]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [189]:
address = 'Toronto, Ontario, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.6534817, -79.3839347.


In [190]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [191]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,0,Café,Grocery Store,Park,Baby Store,Candy Store,Coffee Shop,Playground,Athletics & Sports,Dance Studio,Dog Run
9,West Toronto,0,Bakery,Furniture / Home Store,Park,Café,Smoke Shop,Pharmacy,Bar,Brazilian Restaurant,Bank,Gym / Fitness Center
10,Downtown Toronto,0,Park,Music Venue,Café,Harbor / Marina,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run
20,Central Toronto,0,Park,Dog Run,Breakfast Spot,Bus Line,Gym,Food & Drink Shop,Department Store,Event Space,Electronics Store,Eastern European Restaurant
21,Central Toronto,0,Park,Bus Line,Trail,Diner,Event Space,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run
23,Central Toronto,0,Park,Gym Pool,Garden,Playground,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run
29,Central Toronto,0,Park,Thai Restaurant,Grocery Store,Gym,Dessert Shop,Event Space,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
33,Downtown Toronto,0,Park,Grocery Store,Candy Store,Playground,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run


**The above clusters are highlighted in red color consist of Park, Dog Run, 

In [192]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,1,Spa,Yoga Studio,Diner,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run


**Cluster 1 has only one neighborhood and has mix of fitness and restaurants.**

In [193]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,2,Coffee Shop,Breakfast Spot,Restaurant,Food Truck,Mexican Restaurant,Distribution Center,Pub,Electronics Store,Event Space,Italian Restaurant
1,Downtown Toronto,2,Gym,Italian Restaurant,Gastropub,Hobby Shop,Bubble Tea Shop,Creperie,Burrito Place,Indian Restaurant,Ramen Restaurant,Distribution Center
2,Downtown Toronto,2,Coffee Shop,Café,Theater,Clothing Store,Mexican Restaurant,Electronics Store,Sandwich Place,Restaurant,Burger Joint,Burrito Place
3,Downtown Toronto,2,Gastropub,Italian Restaurant,Japanese Restaurant,Coffee Shop,Café,Restaurant,Beer Bar,Gym,Food Truck,Hostel
4,East Toronto,2,Pub,Gastropub,Trail,Coffee Shop,Bakery,Health Food Store,Yoga Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
5,Downtown Toronto,2,Coffee Shop,Cocktail Bar,Café,Restaurant,Farmers Market,Beer Bar,Gastropub,Basketball Stadium,Museum,Lounge
6,Downtown Toronto,2,Coffee Shop,Japanese Restaurant,Tea Room,American Restaurant,Bubble Tea Shop,Restaurant,Italian Restaurant,Café,Poke Place,Chinese Restaurant
8,Downtown Toronto,2,Coffee Shop,Café,Seafood Restaurant,Restaurant,Hotel,Gym / Fitness Center,Concert Hall,Lounge,Deli / Bodega,Pizza Place
11,West Toronto,2,Bar,Pizza Place,Vietnamese Restaurant,French Restaurant,Park,New American Restaurant,Men's Store,Mac & Cheese Joint,Korean Restaurant,Japanese Restaurant
12,East Toronto,2,Greek Restaurant,Ice Cream Shop,Café,Italian Restaurant,Yoga Studio,Dessert Shop,Diner,Coffee Shop,Cocktail Bar,Pub


**Cluster 2 has many neighborhoods and it has mostly restaurants**

In [194]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,3,Park,Photography Studio,Dessert Shop,Event Space,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run,Distribution Center


**Cluster 3 is similar to cluster 1 has only one neighborhood and consist of everything from park to Disribution center**

In [201]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,West Toronto,4,Park,Residential Building (Apartment / Condo),Comfort Food Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run,Distribution Center


**Cluster 4 is similar to cluster 1 and 3 has only one neighborhood and consist of everything from park to Disribution center**

**Observations**

*   From the map, it is clear that the neighborhood which has top 5 venues as coffee shop, bar, restaurant, cafe are clustered together in sky blue color.
*   Cluster 2 is the perfect place to start a restuarant, cluster 0,3 and 4 are the ideal spots for park and cluster 1 for spa.
